# create_db

Creates and populates a SpatiaLite database.

* 2021-11-25 Changed code so that DATADIR can contain a list of images, or multiple directories, each containing a list of images
* 2021-11-26 Added **execute** function to run commands; raises an error which stops notebook if execution fails
* 2022-10-15 Added parameters tag, improved inline documentation

## spatialite

Obviously, SpatiaLite must be installed:

```
sudo apt-get update -y
sudo apt-get install -y spatialite-bin
```
## other dependancies

* update_trees_table.sql
* create_tree_location_table.sql
* create_views.sql
* create_grid.sql
* create_stats_table.sql

In [19]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.47 ms (started: 2022-10-09 13:39:07 +10:00)


In [20]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import os
import glob
import spatialite
import pandas as pd
import xml.etree.ElementTree as ET

time: 15 ms (started: 2022-10-09 13:39:07 +10:00)


In [21]:
# parameters for papermill
DBPATH = '../output/results.db'
DATADIR = '../rawdata'

time: 7.46 ms (started: 2022-10-09 13:39:07 +10:00)


In [22]:
def execute(command):
    """
    Executes a command line.
    An error is raised if the command fails, stopping the notebook.
    """
    if os.system(command) != 0:
        raise Exception(f'Command failed: {command}')

time: 8.53 ms (started: 2022-10-09 13:39:07 +10:00)


In [23]:
def update_videos_table(video_name):
    sql = 'INSERT INTO videos (name) VALUES (?)'
    cursor.execute(sql, [video_name])
    conn.commit()

time: 8 ms (started: 2022-10-09 13:39:07 +10:00)


In [24]:
def update_frames_table(video_name):
    """
    This function needs to be optimized.
    """  
    
    # Get video_id from videos table    
    sql = (f'SELECT id FROM videos WHERE name = "{video_name}"')
    cursor.execute(sql)
    video_id = cursor.fetchone()[0] 
    
    #print(f'video_id: {video_id}')
    gps_csv_path = f'{DATADIR}/{video_name}/gps-data.csv'
    df = pd.read_csv(gps_csv_path)

    for i, row in df.iterrows():
        lat = row.latitude_adjusted
        lon = row.longitude_adjusted
        geometry = f'TRANSFORM(GeomFromText("POINT({lon} {lat})", 4326), 3857)'
        frame_number = i
        timestamp = row.timestamp        
        sql = f'INSERT INTO frames(video_id,frame_number,time,geometry) VALUES({video_id},{frame_number},"{timestamp}",{geometry});'
        cursor.execute(sql)
        conn.commit()

time: 12.3 ms (started: 2022-10-09 13:39:08 +10:00)


In [25]:
def update_tracks_table(video_name):
    
    # Load dataframe from gps-data.csv
    df = pd.read_csv(f'{DATADIR}/{video_name}/gps-data.csv')

    sql = (f'SELECT id FROM videos WHERE name = "{video_name}"')
    cursor.execute(sql)
    video_id = cursor.fetchone()[0] 

    # Convert dataframe to a geodataframe
    geometry = [Point(xy) for xy in zip(df.longitude_adjusted, df.latitude_adjusted)]
    gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

    # Reproject geometry to UTM (meters)
    gdf = gdf.to_crs('EPSG:3857')

    # Get the LineString
    ls = LineString(gdf.geometry.to_list())

    # Simplify the LineString
    lss = ls.simplify(10)

    sql = f"INSERT INTO tracks VALUES(NULL, '{video_name}', GeomFromText('{lss.wkt}',3857));"
    cursor.execute(sql)
    conn.commit()   

time: 8.81 ms (started: 2022-10-09 13:39:08 +10:00)


In [26]:
def update_trees_and_vcuts_tables(video_name):
    damagedict = {'zero': 0, 'light': 1, 'medium': 2, 'high': 3, 'non_recoverable': 4}
    
    # Get video_id from videos table    
    sql = f'SELECT id FROM videos WHERE name = "{video_name}"'
    cursor.execute(sql)
    video_id = cursor.fetchone()[0] 

    # Iterate through the XML tree, entering records into the trees and vcuts database tables
    cvatxml_path = f'../output/detected_objects.xml'
    root = ET.parse(cvatxml_path).getroot()
    for image in root.iter('image'):
        frame_number = int(image.attrib['id'])
        sql = f'SELECT id FROM frames WHERE video_id={video_id} AND frame_number={frame_number};'
        cursor.execute(sql)

        #frame_id = cursor.fetchone()[0]
        data = cursor.fetchone()
        if data is None:
            print(f'ERROR: No data returned by "{sql}"')
        else:
            frame_id = data[0]
            for box in image.iter('box'):
                damage_index_text = box.attrib['label']
                damage_index = damagedict[damage_index_text]
                xtl = int(float(box.attrib['xtl']))
                ytl = int(float(box.attrib['ytl']))
                xbr = int(float(box.attrib['xbr']))
                ybr = int(float(box.attrib['ybr']))
                geometry = f'GeomFromText("MULTIPOINT({xtl} {ytl},{xbr} {ybr})", -1)'
                sql = f'INSERT OR IGNORE INTO trees(frame_id,damage_index,geometry) VALUES ({frame_id},{damage_index},{geometry});'
                cursor.execute(sql)
                conn.commit()
            for polygon in image.iter('polygon'):
                points = polygon.attrib['points']
                points = points.replace(',', ' ').replace(';', ',') # Reformat points to match WKT for a polygon
                geometry = f'PolyFromText("POLYGON(({points}))",-1)'
                sql = f'INSERT OR IGNORE INTO vcuts(frame_id,geometry) VALUES({frame_id},{geometry});'
                cursor.execute(sql)
                conn.commit()

time: 5.13 ms (started: 2022-10-09 13:39:08 +10:00)


In [27]:
# MAIN

# Delete the database so we can start from scratch

if os.path.exists(DBPATH):
    os.remove(DBPATH)

# Create DB

execute(f'spatialite {DBPATH} < create_tables2.sql')
conn = spatialite.connect(DBPATH)
cursor = conn.cursor()

# Get video list (actually names of folders of images in DATADIR)

video_list = []
for path in glob.glob(f'{DATADIR}/*/'):
    video_list.append(path.split('/')[2])
    
if len(video_list)==0:
    video_list.append(DATADIR)

for video_name in video_list:
    print(video_name)
    print('-- updating videos table')
    update_videos_table(video_name)
    print('-- updating frames table')
    update_frames_table(video_name)
    print('-- updating tracks table')
    update_tracks_table(video_name)
    print('-- updating trees and cuts tables')
    update_trees_and_vcuts_tables(video_name)
cursor.close()
conn.close()

print('Changes to entire database')

print('--updating trees table')
execute(f'spatialite {DBPATH} < update_trees_table.sql')

print('--creating tree location table')
execute(f'spatialite {DBPATH} < create_tree_location_table.sql')

print('--creating views')
execute(f'spatialite {DBPATH} < create_views.sql')

print('--creating grid')
execute(f'spatialite {DBPATH} < create_grid.sql')

print('--creating stats table')
execute(f'spatialite {DBPATH} < create_stats_table.sql')

../rawdata
-- updating videos table
-- updating frames table
-- updating tracks table
-- updating trees and cuts tables
Changes to entire database
--updating trees table
--creating tree location table
--creating views
--creating grid
--creating stats table
time: 4min 42s (started: 2022-10-09 13:39:08 +10:00)
